# Will it rain in Australia tomorrow?

We use the 'Rain in Australia' Kaggle dataset. The datasets contains daily weather observations from Australian weather stations. We train several binary classification models to predict whether or not it will rain the following day. We study accuracy to determine which model performs better. 
For more information see:  
https://www.kaggle.com/jsphyg/weather-dataset-rattle-package/data

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

## Load dataset and perform initial checks

In [2]:
df = pd.read_csv('./weatherAUS.csv')

Check dimensions of dataset

In [3]:
df.shape

(142193, 24)

Check head of dataset

In [4]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


Check info to see column names, their type, and whether we have missing values

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 24 columns):
Date             142193 non-null object
Location         142193 non-null object
MinTemp          141556 non-null float64
MaxTemp          141871 non-null float64
Rainfall         140787 non-null float64
Evaporation      81350 non-null float64
Sunshine         74377 non-null float64
WindGustDir      132863 non-null object
WindGustSpeed    132923 non-null float64
WindDir9am       132180 non-null object
WindDir3pm       138415 non-null object
WindSpeed9am     140845 non-null float64
WindSpeed3pm     139563 non-null float64
Humidity9am      140419 non-null float64
Humidity3pm      138583 non-null float64
Pressure9am      128179 non-null float64
Pressure3pm      128212 non-null float64
Cloud9am         88536 non-null float64
Cloud3pm         85099 non-null float64
Temp9am          141289 non-null float64
Temp3pm          139467 non-null float64
RainToday        140787 non-null obje

In [6]:
df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RISK_MM
count,141556.000000,141871.000000,140787.000000,81350.000000,74377.000000,132923.000000,140845.000000,139563.000000,140419.000000,138583.000000,128179.000000,128212.000000,88536.000000,85099.000000,141289.000000,139467.000000,142193.000000
mean,12.186400,23.226784,2.349974,5.469824,7.624853,39.984292,14.001988,18.637576,68.843810,51.482606,1017.653758,1015.258204,4.437189,4.503167,16.987509,21.687235,2.360682
std,6.403283,7.117618,8.465173,4.188537,3.781525,13.588801,8.893337,8.803345,19.051293,20.797772,7.105476,7.036677,2.887016,2.720633,6.492838,6.937594,8.477969
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-7.200000,-5.400000,0.000000
25%,7.600000,17.900000,0.000000,2.600000,4.900000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.900000,1010.400000,1.000000,2.000000,12.300000,16.600000,0.000000
50%,12.000000,22.600000,0.000000,4.800000,8.500000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.600000,1015.200000,5.000000,5.000000,16.700000,21.100000,0.000000
75%,16.800000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.400000,1020.000000,7.000000,7.000000,21.600000,26.400000,0.800000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000,40.200000,46.700000,371.000000


## Exploratory Data Analysis

## Pre-processing

We observe that we have missing values for most columns. This is how we are going to deal with each of them:  
1. We drop the Date column since we are not interested in it
2. We initially drop the Location column
3. We fill in the missing MinTemp, MaxTemp, Rainfall, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, Temp9am, Temp3pm values with the respective averages since there are only a few missing entries in those columns
4. Evaporation, Sunshine, Cloud9am, Cloud3pm have a lot of missing values (~40%) so we drop these columns altogether
5. WindGustDir, WindDir9am, WindDir3pm, RainToday are categorical features. We drop the rows that have missing values on these columns since they are only a limited amount
7. As suggested on kaggle, we drop the RISK_MM column otherwise it will leak answers to the model during training
8. RainTomorrow is the target and there are no missing values.

In [7]:
df_preprocessed = df.copy()

Implementing 1., 2., 4., 6.

In [8]:
df_preprocessed.drop(['Date','Location','Evaporation','Sunshine','Cloud9am','Cloud3pm','RISK_MM'],axis=1,inplace=True)

Implementing 3.

In [9]:
fill_missing_values = {'MinTemp': df_preprocessed['MinTemp'].mean(),
                       'MaxTemp': df_preprocessed['MaxTemp'].mean(),
                       'Rainfall': df_preprocessed['Rainfall'].mean(),
                       'WindGustSpeed': df_preprocessed['WindGustSpeed'].mean(),
                       'WindSpeed9am': df_preprocessed['WindSpeed9am'].mean(),
                       'WindSpeed3pm': df_preprocessed['WindSpeed3pm'].mean(),
                       'Humidity9am': df_preprocessed['Humidity9am'].mean(),
                       'Humidity3pm': df_preprocessed['Humidity3pm'].mean(),
                       'Pressure9am': df_preprocessed['Pressure9am'].mean(),
                       'Pressure3pm': df_preprocessed['Pressure3pm'].mean(),
                       'Temp9am': df_preprocessed['Temp9am'].mean(),
                       'Temp3pm': df_preprocessed['Temp3pm'].mean() }

df_preprocessed.fillna(fill_missing_values,inplace=True)

Implementing 5.

In [10]:
df_preprocessed.dropna(inplace=True)

Checking final shape of df

In [11]:
df_preprocessed.shape

(123710, 17)

Later improvements:  
1. drop all missing values instead of filling in the averages
2. include Location using dummy variable (maybe PCA to reduce number?)
3. check for outliers and remove them
4. balance target

### Feature selection

In [12]:
inputs = df_preprocessed.drop(['RainTomorrow'],axis=1)

In [13]:
target = df_preprocessed['RainTomorrow']

### Standardize numerical features

In [14]:
scaled_inputs = inputs.copy()
num_features = scaled_inputs.drop(['WindGustDir','WindDir9am','WindDir3pm','RainToday'],axis=1)
num_features_col = num_features.columns.values

scaler = StandardScaler()
scaler.fit(num_features.values)
num_features = scaler.transform(num_features.values)
scaled_inputs[num_features_col] = num_features

In [15]:
scaled_inputs.head()

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday
0,0.154881,-0.079770,-0.208215,W,0.247578,W,WNW,0.600477,0.562994,0.175806,-1.384691,-1.436219,-1.182863,-0.056940,-0.013664,No
1,-0.787513,0.225537,-0.278572,WNW,0.247578,NNW,WSW,-1.325551,0.329649,-1.247089,-1.240733,-1.007585,-1.078445,-0.010629,0.343181,No
2,0.076348,0.308803,-0.278572,WSW,0.396959,W,WSW,0.480100,0.796339,-1.563287,-1.000804,-1.450999,-0.944193,0.575985,0.186169,No
3,-0.504795,0.627987,-0.278572,NE,-1.246232,SE,E,-0.482914,-1.187093,-1.194389,-1.672606,0.027050,-0.332599,0.128306,0.657205,No
4,0.798850,1.224723,-0.161311,W,0.023507,ENE,NW,-0.964421,0.096304,0.755504,-0.856846,-0.978024,-1.346950,0.081994,1.113967,No


### Create dummy variables for categorial features and target

Check what possible values the categorical features can have

In [16]:
print('RainToday = ',scaled_inputs['RainToday'].unique())
print('RainTomorrow = ',target.unique())
print('WindGustDir = ',scaled_inputs['WindGustDir'].unique())
print('WindDir9am = ',scaled_inputs['WindDir9am'].unique())
print('WindDir3pm = ',scaled_inputs['WindDir3pm'].unique())

RainToday =  ['No' 'Yes']
RainTomorrow =  ['No' 'Yes']
WindGustDir =  ['W' 'WNW' 'WSW' 'NE' 'NNW' 'N' 'NNE' 'SW' 'ENE' 'SSE' 'S' 'NW' 'SE' 'ESE'
 'E' 'SSW']
WindDir9am =  ['W' 'NNW' 'SE' 'ENE' 'SW' 'SSE' 'S' 'NE' 'SSW' 'N' 'WSW' 'ESE' 'E' 'NW'
 'WNW' 'NNE']
WindDir3pm =  ['WNW' 'WSW' 'E' 'NW' 'W' 'SSE' 'ESE' 'ENE' 'NNW' 'SSW' 'SW' 'SE' 'N' 'S'
 'NNE' 'NE']


Map Yes and No in RainToday and RainTomorrow to 1 and 0

In [17]:
scaled_inputs['RainToday'].replace({'No': 0, 'Yes': 1},inplace = True)
target.replace({'No': 0, 'Yes': 1},inplace = True)

Create dummy variables for the other three features and drop first

In [18]:
scaled_inputs = pd.get_dummies(scaled_inputs,columns=['WindGustDir','WindDir9am','WindDir3pm'],drop_first=True)

In [19]:
scaled_inputs.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,0.154881,-0.079770,-0.208215,0.247578,0.600477,0.562994,0.175806,-1.384691,-1.436219,-1.182863,...,0,0,0,0,0,0,0,0,1,0
1,-0.787513,0.225537,-0.278572,0.247578,-1.325551,0.329649,-1.247089,-1.240733,-1.007585,-1.078445,...,0,0,0,0,0,0,0,0,0,1
2,0.076348,0.308803,-0.278572,0.396959,0.480100,0.796339,-1.563287,-1.000804,-1.450999,-0.944193,...,0,0,0,0,0,0,0,0,0,1
3,-0.504795,0.627987,-0.278572,-1.246232,-0.482914,-1.187093,-1.194389,-1.672606,0.027050,-0.332599,...,0,0,0,0,0,0,0,0,0,0
4,0.798850,1.224723,-0.161311,0.023507,-0.964421,0.096304,0.755504,-0.856846,-0.978024,-1.346950,...,0,1,0,0,0,0,0,0,0,0


Later improvements:
1. select important features using SelectKBest

## Logistic regression

### Train-test split

In [20]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, target, test_size = 0.2, random_state = 20)

### Training the model

In [21]:
reg = LogisticRegression(solver='lbfgs')
reg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Prediction of the model on train set

In [22]:
y_train_reg_pred = reg.predict(x_train)

Accuracy on train set

In [23]:
reg.score(x_train,y_train)

0.845596556462695

Confusion matrix on train set

In [24]:
confusion_matrix(y_train, y_train_reg_pred)

array([[72910,  4130],
       [11151, 10777]])

### Finding coefficients and intercept

In [25]:
reg.intercept_

array([-2.18944791])

In [26]:
reg.coef_

array([[ 0.26786096, -0.31593985,  0.06783263,  0.74431582, -0.10920092,
        -0.27031062,  0.05903793,  1.29541449,  0.79762088, -1.13568605,
         0.05813875,  0.04056697,  0.57805557, -0.07772045,  0.11666432,
         0.12748738, -0.19058975, -0.08939316,  0.16722247,  0.1090108 ,
         0.10144843,  0.18924261,  0.19158754,  0.11524421,  0.20673768,
         0.15370444,  0.17595761,  0.17961123,  0.29803956, -0.16381704,
         0.31151744,  0.34163176,  0.50661998,  0.0429716 , -0.01864135,
        -0.18625252, -0.17883498, -0.13356803, -0.12087521,  0.05276499,
        -0.00963201,  0.02896887,  0.00392846, -0.0713448 , -0.00276538,
         0.24616964, -0.26645589,  0.05527108,  0.39130334,  0.37023818,
         0.05454923,  0.02199238, -0.07036029,  0.05239917, -0.0460998 ,
         0.07733052,  0.25287012,  0.01937473]])

In [64]:
summary_table = pd.DataFrame(columns=['Feature'], data = scaled_inputs.columns.values)
summary_table['Coefficient'] = np.transpose(reg.coef_)

### Interpreting the coefficients

In [65]:
summary_table['Odds Ratio'] = np.exp(summary_table.Coefficient)
summary_table.sort_values('Odds Ratio', ascending=False)

,Feature,Coefficient,Odds Ratio
7,Humidity3pm,1.295414,3.652510
8,Pressure9am,0.797621,2.220252
3,WindGustSpeed,0.744316,2.105001
12,RainToday,0.578056,1.782569
32,WindDir9am_NNE,0.506620,1.659672
48,WindDir3pm_NNW,0.391303,1.478907
49,WindDir3pm_NW,0.370238,1.448079
31,WindDir9am_NE,0.341632,1.407242
30,WindDir9am_N,0.311517,1.365496
28,WindDir9am_ENE,0.298040,1.347215


Comment on interpretation of coefficients

### Testing the model

Accuracy on test set

In [29]:
reg.score(x_test,y_test)

0.8473041791286072

In [30]:
y_test_reg_pred = reg.predict(x_test)
confusion_matrix(y_test, y_test_reg_pred)

array([[18276,  1002],
       [ 2776,  2688]])

Predicted probabilities on test set

In [31]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.84737681, 0.15262319],
       [0.81063107, 0.18936893],
       [0.99142831, 0.00857169],
       ...,
       [0.77962129, 0.22037871],
       [0.67353827, 0.32646173],
       [0.38634774, 0.61365226]])

## Random Forest Classifier

### Training the model

In [32]:
rf = RandomForestClassifier(n_estimators = 100, max_depth = 4, random_state = 20)
rf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=20, verbose=0,
                       warm_start=False)

In [33]:
y_train_rf_pred = rf.predict(x_train)

In [34]:
rf.score(x_train,y_train)

0.8251353973001374

In [35]:
confusion_matrix(y_train, y_train_rf_pred)

array([[75801,  1239],
       [16067,  5861]])

### Testing the model

In [36]:
rf.score(x_test,y_test)

0.8249939374343223

In [37]:
y_test_rf_pred = rf.predict(x_test)
confusion_matrix(y_test, y_test_rf_pred)

array([[18962,   316],
       [ 4014,  1450]])

## Decision Tree Classifier

### Training the model

In [38]:
dt = DecisionTreeClassifier(random_state = 20)
dt.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=20, splitter='best')

In [39]:
y_train_dt_pred = dt.predict(x_train)

In [40]:
dt.score(x_train,y_train)

1.0

In [41]:
confusion_matrix(y_train, y_train_dt_pred)

array([[77040,     0],
       [    0, 21928]])

### Testing the model

In [42]:
dt.score(x_test,y_test)

0.7924581682968233

In [43]:
y_test_dt_pred = dt.predict(x_test)
confusion_matrix(y_test, y_test_dt_pred)

array([[16627,  2651],
       [ 2484,  2980]])

## Neural Network using TensorFlow 2

### Further split train set into train and validation sets. Overall split: 64 (train) - 16 (val) - 20 (test)

In [45]:
x_train_nn, x_val_nn, y_train_nn, y_val_nn = train_test_split(x_train, y_train, test_size=0.2)

Convert dataframes into numpy arrays for tensorflow

In [46]:
# train
train_inputs, train_target = x_train_nn.values, y_train_nn.values
# validation
validation_inputs, validation_target = x_val_nn.values, y_val_nn.values
# test
test_inputs, test_target = x_test.values, y_test.values

### Define and train neural network
Outline of the network:  
1. Two hidden layers with 256 nodes each. We choose relu for the 1st and 2nd layers and softmax for the output layer to obtain probabilities for the two classes
2. We choose adam as the optimizer  
3. We choose sparse_categorical_crossentropy as the loss function since this is a classification problem
4. We choose accuracy as the metric to measure the performance of the NN
5. We use a validation set with early stopping (with patience=2) to prevent overfitting of the train set  
6. Finally, we fix the batch size to be 100 and the max number of epochs to be 100

In [96]:
input_size = len(x_train_nn.columns)
output_size = 2

hidden_layer_size = 256

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_target,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_target),
          verbose = 2
          )  

Train on 79174 samples, validate on 19794 samples
Epoch 1/100
79174/79174 - 6s - loss: 0.3574 - accuracy: 0.8449 - val_loss: 0.3495 - val_accuracy: 0.8490
Epoch 2/100
79174/79174 - 7s - loss: 0.3395 - accuracy: 0.8529 - val_loss: 0.3424 - val_accuracy: 0.8501
Epoch 3/100
79174/79174 - 6s - loss: 0.3329 - accuracy: 0.8561 - val_loss: 0.3421 - val_accuracy: 0.8522
Epoch 4/100
79174/79174 - 7s - loss: 0.3280 - accuracy: 0.8583 - val_loss: 0.3373 - val_accuracy: 0.8526
Epoch 5/100
79174/79174 - 6s - loss: 0.3243 - accuracy: 0.8598 - val_loss: 0.3399 - val_accuracy: 0.8515
Epoch 6/100
79174/79174 - 6s - loss: 0.3197 - accuracy: 0.8618 - val_loss: 0.3416 - val_accuracy: 0.8522


### Testing the model

In [97]:
y_test_nn_pred_proba = model.predict(test_inputs)
y_test_nn_pred = model.predict_classes(test_inputs)

In [98]:
conf_mat = confusion_matrix(test_target, y_test_nn_pred)
conf_mat

array([[17878,  1400],
       [ 2200,  3264]])

In [99]:
nn_score = (conf_mat[0,0]+conf_mat[1,1])/len(test_target)
nn_score

0.8544984237329237

In [102]:
len(x_train_nn.columns)

58

In [44]:
#df_copy = df.copy()
#df_copy['RainTomorrow'] = df_copy['RainTomorrow'].astype(bool)


#df_copy.groupby('Location').sum()